In [ ]:
#import libraries
import time
import ipywidgets as widgets
from IPython.display import display, Image, clear_output, HTML
from jupyter_ui_poll import ui_events
import requests
from bs4 import BeautifulSoup
import json

# function for send test results to google form
def send_to_google_form(data_dict, form_url):
        form_id = form_url[34:90]
        view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
        post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

        page = requests.get(view_form_url)
        content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
        content = content.text[27:-1]
        result = json.loads(content)[1][1]
        form_dict = {}
    
        loaded_all = True
        for item in result:
            if item[1] not in data_dict:
                print(f"Form item {item[1]} not found. Data not uploaded.")
                loaded_all = False
                return False
            form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
        post_result = requests.post(post_form_url, data=form_dict)
        return post_result.ok


# function for run ANS test
def test1():
    # function for waiting 
    event_info = {
    'type': '',
    'description': '',
    'time': -1
    }

    def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):
        start_wait = time.time()
        event_info['type'] = ""
        event_info['description'] = ""
        event_info['time'] = -1

        n_proc = int(max_rate*interval)+1
        
        with ui_events() as ui_poll:
            keep_looping = True
            while keep_looping==True:
                ui_poll(n_proc)

                if (timeout != -1) and (time.time() > start_wait + timeout):
                    keep_looping = False
                
                if allow_interupt==True and event_info['description']!="":
                    keep_looping = False
            
                time.sleep(interval)
        return event_info
    
    # function for bottons
    def register_event(btn):
        event_info['type'] = "click"
        event_info['description'] = btn.description
        event_info['time'] = time.time()
        return
    
    # function for displaying image
    def display_img(img_file):
        style_str = f'width: 500px'
        html_out = HTML(f"<img style='{style_str}' src={img_file}></img>")
        display(html_out)

    # function for running the test and compare results with correct answer
    def which_more(img_file, right_answer):
        time.sleep(1.5)
        display_img(img_file) 
        time.sleep(0.75)  
        clear_output(wait=False)
        # buttons
        btn1 = widgets.Button(description="Left")
        btn2 = widgets.Button(description="Right")
        btn1.on_click(register_event)
        btn2.on_click(register_event)
        myhtml1 = HTML("<h1>You have 3 seconds to answer</h1>")
        display(myhtml1)

        panel = widgets.HBox([btn1, btn2])
        display(panel)
        result = wait_for_event(timeout=3)
        clear_output()
        
        if result['description']!="":
            print(f"User clicked: {result['description']}")
        else:
            print("User did not click in time")

        score = 0
    
        if event_info['description'] == right_answer:
            print("Well done!")
            score = 1
        else:
            print(f"Sorry the answer was {right_answer}")
    
        return score
    # define the correct answer
    files = ["1.png","2.png","3.png","4.png","5.png","6.png","7.png","8.png","9.png","10.png","11.png","12.png","13.png","14.png","15.png","16.png","17.png","18.png","19.png","20.png","21.png","22.png","23.png","24.png","25.png","26.png","27.png","28.png","29.png","30.png","31.png","32.png","33.png","34.png","35.png","36.png","37.png","38.png","39.png","40.png","41.png","42.png","43.png","44.png","45.png","46.png","47.png","48.png","49.png","50.png","51.png","52.png","53.png","54.png","55.png","56.png","57.png","58.png","59.png","60.png","61.png","62.png","63.png","64.png"]
    answers = ["Right","Right","Left","Left","Right","Right","Left","Right","Left","Left","Left","Left","Right","Left","Right","Right","Left","Right","Left","Right","Right","Right","Right","Right","Right","Left","Right","Right","Right","Right","Right","Left","Right","Right","Right","Left","Right","left","Left","Left","Left","Left","Right","Left","Right","Left","Right","Left","Right","Left","Left","Left","Right","Right","Left","Left","Left","Left","Right","Left","Right","Right","Right","Left","Left","Right","Right"]
    print("Welcome to the ANS testing")

    
    time.sleep(1)
    print("From the following test you are expecting to estimate which circle contains more small circles within,please choose left or right:")
    time.sleep(3)
    print("You have 3 seconds to answer each question, the whole test have 64 questions taking about 3 and a half minutes")
    time.sleep(3)
    total = 0
    n_correct = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    start_time = time.time()
    
    # get output for time taken, total score, and score for individual questions
    for i in range(len(files)):
        image = f"dots/{files[i]}"
        score = which_more(image, answers[i])
        total += score
        n_correct[i] += score
        end_time = time.time()
        time_taken = round(end_time - start_time)
    
    return (time_taken, total, n_correct)

# Disclaimer
data_consent_info = """DATA CONSENT INFORMATION:
Please read:
we wish to record your response data
to an anonymised public data repository.
Your data will be used for educational teaching purposes
practising data analysis and visualisation.
Please type yes in the box below if you consent to the upload."""
print(data_consent_info)

result = input("> ")
if result == "yes":
    print("Thanks for your participation.")
    print("Please contact philip.lewis@ucl.ac.uk")
    print("If you have any questions or concerns")
    print("regarding the stored results.")
else:
    raise(Exception("User did not consent to continue test."))

id_instructions = """

Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:
- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise
     then your unique identifer would be CBTC
"""

print(id_instructions)

user_id = input("> ")
print("User entered id:", user_id)

name = input("Please enter your name")
age = input("Please enter your age")
time, final, result_list = test1()
print("You have completed the test! Well done!")

# Get participants'consent of data collection and transfer to google form
data_dict = {
    'User_ID': user_id,
    'Name': name,
    'Age': age,
    'Time':time,
    'R1': final,
    'R2': result_list
}

form_url = "https://docs.google.com/forms/d/e/1FAIpQLSd4Dc0_aVbGlzdei-zGdBnJ81BXmkonYcX8_V7qU2h9Zs6KhA/viewform"
send_to_google_form(data_dict, form_url)




DATA CONSENT INFORMATION:
Please read:
we wish to record your response data
to an anonymised public data repository.
Your data will be used for educational teaching purposes
practising data analysis and visualisation.
Please type yes in the box below if you consent to the upload.


>  yes


Thanks for your participation.
Please contact philip.lewis@ucl.ac.uk
If you have any questions or concerns
regarding the stored results.


Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:
- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise
     then your unique identifer would be CBTC

